# Load Dataset

## Load Interactions Dataset

In [1]:
with open("dataset/amazon-custom-books/amazon-custom-books.inter", "r", encoding="utf-8") as f:
    source_dataset = [line.split("\t") for line in f.read().split("\n")][1:-1]

with open("dataset/amazon-custom-movies-tv/amazon-custom-movies-tv.inter", "r", encoding="utf-8") as f:
    target_dataset = [line.split("\t") for line in f.read().split("\n")][1:-1]

In [2]:
source_users = list(set([row[0] for row in source_dataset]))
source_items = list(set([row[1] for row in source_dataset]))

target_users = list(set([row[0] for row in target_dataset]))
target_items = list(set([row[1] for row in target_dataset]))

In [3]:
all_users = list(set(source_users + target_users))
all_items = list(set(source_items + target_items))

n_users = len(all_users)
n_source_items = len(source_items)
n_target_items = len(target_items)
n_items = len(all_items)

In [4]:
print("source_interactions:", len(source_dataset))
print("target_interactions:", len(target_dataset))
print()
print("n_users:", n_users)
print("n_source_users:", len(source_users))
print("n_target_users:", len(target_users))
print()
print("n_source_items:", n_source_items)
print("n_target_items:", n_target_items)
print("n_items:", n_items)
print()
print("overlap_users:", len(source_users)+len(target_users)-n_users)
print("overlap_items:", n_source_items+n_target_items-n_items)

source_interactions: 104080
target_interactions: 120985

n_users: 3859
n_source_users: 2329
n_target_users: 2346

n_source_items: 4883
n_target_items: 4929
n_items: 9812

overlap_users: 816
overlap_items: 0


In [5]:
user_dict = {user: idx for idx, user in enumerate(all_users)}
source_item_dict = {item: idx for idx, item in enumerate(source_items)}
target_item_dict = {item: idx for idx, item in enumerate(target_items)}
item_dict = {item: idx for idx, item in enumerate(all_items)}

In [6]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# raw dataset to torch dataset with integrated user ids but separate item ids

tensor_source_users = torch.Tensor(np.array([user_dict[row[0]] for row in source_dataset]))
tensor_source_items = torch.Tensor(np.array([source_item_dict[row[1]] for row in source_dataset]))
tensor_source_ratings = torch.Tensor(np.array([float(row[2]) for row in source_dataset]))

tensor_target_users = torch.Tensor(np.array([user_dict[row[0]] for row in target_dataset]))
tensor_target_items = torch.Tensor(np.array([target_item_dict[row[1]] for row in target_dataset]))
tensor_target_ratings = torch.Tensor(np.array([float(row[2]) for row in target_dataset]))

source_dataset_torch = TensorDataset(tensor_source_users, tensor_source_items, tensor_source_ratings)
target_dataset_torch = TensorDataset(tensor_target_users, tensor_target_items, tensor_target_ratings)

## Load Item Vectors

In [7]:
with open("dataset/amazon-custom-books/amazon-custom-books.embed", "r", encoding="utf-8") as f:
    source_item_embed_file = [line.split("\t") for line in f.read().split("\n")][1:-1]

with open("dataset/amazon-custom-movies-tv/amazon-custom-movies-tv.embed", "r", encoding="utf-8") as f:
    target_item_embed_file = [line.split("\t") for line in f.read().split("\n")][1:-1]

In [8]:
source_item_embed_dict = {line[0]: list(map(float, line[1].replace("[", "").replace("]", "").split(","))) for line in source_item_embed_file}
target_item_embed_dict = {line[0]: list(map(float, line[1].replace("[", "").replace("]", "").split(","))) for line in target_item_embed_file}

In [9]:
reverse_source_item_dict = {source_item_dict[item]: item for item in source_items}
reverse_target_item_dict = {target_item_dict[item]: item for item in target_items}

source_item_feature_vectors = np.array([source_item_embed_dict[reverse_source_item_dict[idx]] for idx in range(n_source_items)])
target_item_feature_vectors = np.array([target_item_embed_dict[reverse_target_item_dict[idx]] for idx in range(n_target_items)])

# Model Engineering

In [ ]:
import math
from typing import Literal

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

class CMF(nn.Module):
    """
    Collective Matrix Factorization for two domains sharing user embeddings.

    Assumes datasets yield (user_idx, item_idx, rating) with 0-based integer indices.
    """
    def __init__(
        self,
        n_users: int,
        n_items_source: int,
        n_items_target: int,
        emb_dim: int = 64,
        use_bias: bool = True,
    ):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb_source = nn.Embedding(n_items_source, emb_dim)
        self.item_emb_target = nn.Embedding(n_items_target, emb_dim)

        self.use_bias = use_bias
        if use_bias:
            self.user_bias = nn.Embedding(n_users, 1)
            self.item_bias_source = nn.Embedding(n_items_source, 1)
            self.item_bias_target = nn.Embedding(n_items_target, 1)
            self.global_bias = nn.Parameter(torch.zeros(1))
        else:
            # placeholders to simplify forward
            self.register_buffer("user_bias", torch.zeros(n_users, 1))
            self.register_buffer("item_bias_source", torch.zeros(n_items_source, 1))
            self.register_buffer("item_bias_target", torch.zeros(n_items_target, 1))
            self.register_buffer("global_bias", torch.zeros(1))

        # init
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.user_emb.weight, 0, 0.1)
        nn.init.normal_(self.item_emb_source.weight, 0, 0.1)
        nn.init.normal_(self.item_emb_target.weight, 0, 0.1)
        if self.use_bias:
            nn.init.zeros_(self.user_bias.weight)
            nn.init.zeros_(self.item_bias_source.weight)
            nn.init.zeros_(self.item_bias_target.weight)
            nn.init.zeros_(self.global_bias)

    def predict(self, users: torch.LongTensor, items: torch.LongTensor, domain: Literal["source", "target"]):
        """
        Predict ratings for a batch given domain.
        domain: "source" or "target"
        """
        u_emb = self.user_emb(users)
        if domain == "source":
            i_emb = self.item_emb_source(items)
            if self.use_bias:
                ub = self.user_bias(users).squeeze(-1)
                ib = self.item_bias_source(items).squeeze(-1)
            else:
                ub = 0.0
                ib = 0.0
        elif domain == "target":
            i_emb = self.item_emb_target(items)
            if self.use_bias:
                ub = self.user_bias(users).squeeze(-1)
                ib = self.item_bias_target(items).squeeze(-1)
            else:
                ub = 0.0
                ib = 0.0
        else:
            raise ValueError("domain must be 'source' or 'target'")

        dot = (u_emb * i_emb).sum(dim=-1)
        return dot + (ub + ib + self.global_bias).squeeze()  # shape (batch,)

    def forward(self, users: torch.LongTensor, items: torch.LongTensor, domain: Literal["source", "target"]):
        return self.predict(users, items, domain)


def train_cmf(
    model: CMF,
    source_loader: DataLoader,
    target_loader: DataLoader,
    epochs: int = 10,
    lr: float = 1e-3,
    weight_decay: float = 0.0,
    device: torch.device = None,
    report_every: int = 1,
):
    """
    Jointly train on source and target loaders. Each loader yields (user, item, rating).
    """
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()

    for ep in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        n_batches = 0

        # Train on source domain
        for users, items, ratings in source_loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)

            preds = model(users, items, domain="source")
            loss = criterion(preds, ratings)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            n_batches += 1

        # Train on target domain
        for users, items, ratings in target_loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)

            preds = model(users, items, domain="target")
            loss = criterion(preds, ratings)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            n_batches += 1

        if ep % report_every == 0:
            avg_loss = total_loss / max(1, n_batches)
            print(f"Epoch {ep}/{epochs} — avg MSE: {avg_loss:.6f}")

    return model


def evaluate_cmf(model: CMF, loader: DataLoader, domain: Literal["source", "target"], device: torch.device = None):
    """
    Return RMSE on provided loader (domain indicates which item embeddings to use).
    """
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    model.to(device)
    model.eval()
    se = 0.0
    n = 0
    with torch.no_grad():
        for users, items, ratings in loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)
            preds = model(users, items, domain=domain)
            se += ((preds - ratings) ** 2).sum().item()
            n += ratings.numel()
    rmse = math.sqrt(se / n) if n > 0 else float("nan")
    return rmse

# Train Model

In [11]:
source_loader = DataLoader(source_dataset_torch, batch_size=1024, shuffle=True)
target_loader = DataLoader(target_dataset_torch, batch_size=1024, shuffle=True)

In [ ]:
model = CMF(n_users, n_source_items, n_target_items, emb_dim=64, use_bias=False)

train_cmf(model, source_loader, target_loader, epochs=50, lr=1e-3)

rmse_target = evaluate_cmf(model, target_loader, domain="target")

print("Target RMSE:", rmse_target)

Epoch 1/50 — avg MSE: 18.636065
Epoch 2/50 — avg MSE: 18.415414
Epoch 3/50 — avg MSE: 18.032449
Epoch 4/50 — avg MSE: 17.013471
Epoch 5/50 — avg MSE: 14.360445
Epoch 6/50 — avg MSE: 9.993300
Epoch 7/50 — avg MSE: 5.951675
Epoch 8/50 — avg MSE: 3.436597
Epoch 9/50 — avg MSE: 2.110143
Epoch 10/50 — avg MSE: 1.428026
Epoch 11/50 — avg MSE: 1.059915
Epoch 12/50 — avg MSE: 0.852664
Epoch 13/50 — avg MSE: 0.728021
Epoch 14/50 — avg MSE: 0.647738
Epoch 15/50 — avg MSE: 0.589273
Epoch 16/50 — avg MSE: 0.544644
Epoch 17/50 — avg MSE: 0.507299
Epoch 18/50 — avg MSE: 0.473792
Epoch 19/50 — avg MSE: 0.442900
Epoch 20/50 — avg MSE: 0.413502
Epoch 21/50 — avg MSE: 0.385857
Epoch 22/50 — avg MSE: 0.358780
Epoch 23/50 — avg MSE: 0.332561
Epoch 24/50 — avg MSE: 0.307609
Epoch 25/50 — avg MSE: 0.283845
Epoch 26/50 — avg MSE: 0.260883
Epoch 27/50 — avg MSE: 0.239688
Epoch 28/50 — avg MSE: 0.219412
Epoch 29/50 — avg MSE: 0.200611
Epoch 30/50 — avg MSE: 0.183223
Epoch 31/50 — avg MSE: 0.167148
Epoch 32/50 

In [16]:
preds = model.predict(tensor_target_users.long(), tensor_target_items.long(), 'target')

In [17]:
bin_preds = [int(e >= 3) for e in preds]

In [18]:
bin_ratings = [int(e >= 3) for e in tensor_target_ratings]

In [19]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

acc = accuracy_score(bin_ratings, bin_preds)
conf = precision_recall_fscore_support(bin_ratings, bin_preds, average='binary')

In [20]:
print("acc:", acc)
print("prec:", conf[0])
print("recall:", conf[1])
print("f1:", conf[2])

acc: 0.9510435177914618
prec: 0.9997985708119514
recall: 0.9464117091595845
f1: 0.9723729074448088


# NCF

In [ ]:
import math
import torch

# Neural collaborative filtering (NCF) for cross-domain recommendation
# Uses precomputed item feature vectors for source / target domains.
# Expects these variables to exist in the notebook:
#   - user_dict (dict): maps user_id -> user_idx (0-based)
#   - n_users (int)
#   - source_item_feature_vectors (np.array or torch.Tensor)  # shape (n_source_items, feat_dim)
#   - target_item_feature_vectors (np.array or torch.Tensor)  # shape (n_target_items, feat_dim)
#   - source_loader, target_loader (torch.utils.data.DataLoader) or source_dataset_torch/target_dataset_torch
# If prebuilt loaders are not available, you can create them from source_dataset_torch / target_dataset_torch.

import torch.nn as nn

class NCF(nn.Module):
    """
    Neural Collaborative Filtering that concatenates a learned user embedding with provided
    item feature vectors (from source/target) and passes them through an MLP to predict rating.
    """
    def __init__(
        self,
        n_users: int,
        source_item_features,  # numpy array or torch.Tensor (n_source_items, feat_dim)
        target_item_features,  # numpy array or torch.Tensor (n_target_items, feat_dim)
        user_emb_dim: int = 64,
        mlp_layers: list = (128, 64, 32),
        dropout: float = 0.2,
        freeze_item_features: bool = True,
    ):
        super().__init__()
        # user embedding
        self.user_emb = nn.Embedding(n_users, user_emb_dim)

        # item features: convert to tensors and make Embedding-like lookup via from_pretrained
        src_feat = torch.as_tensor(source_item_features, dtype=torch.float32)
        tgt_feat = torch.as_tensor(target_item_features, dtype=torch.float32)
        self.src_feat_dim = src_feat.shape[1]
        self.tgt_feat_dim = tgt_feat.shape[1]

        self.item_feat_src = nn.Embedding.from_pretrained(src_feat, freeze=freeze_item_features)
        self.item_feat_tgt = nn.Embedding.from_pretrained(tgt_feat, freeze=freeze_item_features)

        # MLP that takes concat(user_emb, item_feat) -> rating scalar
        input_dim_src = user_emb_dim + self.src_feat_dim
        input_dim_tgt = user_emb_dim + self.tgt_feat_dim
        # For simplicity use same MLP architecture but different first layer shapes handled at forward

        # Build MLP for source domain and target domain separately if feature dims differ
        def build_mlp(input_dim):
            layers = []
            prev_dim = input_dim

            for hidden_dim in mlp_layers:
                layers.append(nn.Linear(prev_dim, hidden_dim))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(p=dropout))
                prev_dim = hidden_dim

            layers.append(nn.Linear(prev_dim, 1))  # final scalar
            return nn.Sequential(*layers)

        self.mlp_src = build_mlp(input_dim_src)
        # reuse architecture for target but with target input dim
        self.mlp_tgt = build_mlp(input_dim_tgt)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.user_emb.weight, 0, 0.1)

        # pretrained item features already set; MLP init
        for m in self.mlp_src:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

        for m in self.mlp_tgt:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, users: torch.LongTensor, items: torch.LongTensor, domain: str):
        """
        users: (batch,)
        items: (batch,) -- item indices local to domain (0..n_items_domain-1)
        domain: "source" or "target"
        returns: (batch,) predicted scalar ratings
        """
        u = self.user_emb(users)  # (batch, user_emb_dim)

        if domain == "source":
            i_feat = self.item_feat_src(items)  # (batch, feat_dim)
            x = torch.cat([u, i_feat], dim=-1)
            out = self.mlp_src(x).squeeze(-1)

        elif domain == "target":
            i_feat = self.item_feat_tgt(items)
            x = torch.cat([u, i_feat], dim=-1)
            out = self.mlp_tgt(x).squeeze(-1)

        else:
            raise ValueError("domain must be 'source' or 'target'")
        
        return out

# Training / evaluation helpers
def train_ncf(
    model: nn.Module,
    source_loader,
    target_loader,
    epochs: int = 10,
    lr: float = 1e-3,
    weight_decay: float = 0.0,
    device: torch.device = None,
    report_every: int = 1,
):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = nn.MSELoss()

    for ep in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        n_batches = 0

        for users, items, ratings in source_loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)

            preds = model(users, items, domain="source")
            loss = loss_fn(preds, ratings)
            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            n_batches += 1

        for users, items, ratings in target_loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)

            preds = model(users, items, domain="target")
            loss = loss_fn(preds, ratings)
            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            n_batches += 1

        if ep % report_every == 0:
            print(f"Epoch {ep}/{epochs} avg MSE: {total_loss / max(1, n_batches):.6f}")

    return model

def evaluate_ncf(model: nn.Module, loader, domain: str, device: torch.device = None):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    model.to(device)
    model.eval()
    se = 0.0
    n = 0
    with torch.no_grad():
        for users, items, ratings in loader:
            users = users.long().to(device)
            items = items.long().to(device)
            ratings = ratings.float().to(device)
            preds = model(users, items, domain=domain)
            se += ((preds - ratings) ** 2).sum().item()
            n += ratings.numel()
    rmse = math.sqrt(se / n) if n > 0 else float("nan")
    return rmse

In [ ]:
# ...existing code...
class NCF(nn.Module):
    """
    Neural Collaborative Filtering that concatenates a learned user embedding with provided
    item feature vectors (from source/target) and passes them through an MLP to predict rating.

    This version shares a base user embedding across domains and applies small domain-specific
    adapters (learned transforms) so the model can learn domain-specific user behavior while
    still sharing signal across domains (helpful for overlapping users).
    """
    def __init__(
        self,
        n_users: int,
        source_item_features,  # numpy array or torch.Tensor (n_source_items, feat_dim)
        target_item_features,  # numpy array or torch.Tensor (n_target_items, feat_dim)
        user_emb_dim: int = 64,
        mlp_layers: list = (128, 64, 32),
        dropout: float = 0.2,
        freeze_item_features: bool = True,
        adapter_hidden: int = None,  # if set, use a small hidden layer in adapters
    ):
        super().__init__()
        # shared base user embedding (size = total unique users across both domains)
        self.user_emb = nn.Embedding(n_users, user_emb_dim)

        # lightweight domain adapters: map base user embedding -> domain-specific user embedding
        if adapter_hidden is None:
            # single linear adapter
            self.user_adapter_src = nn.Linear(user_emb_dim, user_emb_dim)
            self.user_adapter_tgt = nn.Linear(user_emb_dim, user_emb_dim)
        else:
            self.user_adapter_src = nn.Sequential(
                nn.Linear(user_emb_dim, adapter_hidden),
                nn.ReLU(),
                nn.Linear(adapter_hidden, user_emb_dim),
            )
            self.user_adapter_tgt = nn.Sequential(
                nn.Linear(user_emb_dim, adapter_hidden),
                nn.ReLU(),
                nn.Linear(adapter_hidden, user_emb_dim),
            )

        # item features: convert to tensors and make Embedding-like lookup via from_pretrained
        src_feat = torch.as_tensor(source_item_features, dtype=torch.float32)
        tgt_feat = torch.as_tensor(target_item_features, dtype=torch.float32)
        self.src_feat_dim = src_feat.shape[1]
        self.tgt_feat_dim = tgt_feat.shape[1]

        self.item_feat_src = nn.Embedding.from_pretrained(src_feat, freeze=freeze_item_features)
        self.item_feat_tgt = nn.Embedding.from_pretrained(tgt_feat, freeze=freeze_item_features)

        # MLPs for each domain
        input_dim_src = user_emb_dim + self.src_feat_dim
        input_dim_tgt = user_emb_dim + self.tgt_feat_dim

        def build_mlp(input_dim):
            layers = []
            prev = input_dim
            for h in mlp_layers:
                layers.append(nn.Linear(prev, h))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(p=dropout))
                prev = h
            layers.append(nn.Linear(prev, 1))
            return nn.Sequential(*layers)

        self.mlp_src = build_mlp(input_dim_src)
        self.mlp_tgt = build_mlp(input_dim_tgt)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.user_emb.weight, 0, 0.1)
        # adapter init
        def init_adapter(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        self.user_adapter_src.apply(init_adapter)
        self.user_adapter_tgt.apply(init_adapter)
        # MLP init
        for m in self.mlp_src:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)
        for m in self.mlp_tgt:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, users: torch.LongTensor, items: torch.LongTensor, domain: str):
        """
        users: (batch,)
        items: (batch,) -- item indices local to domain (0..n_items_domain-1)
        domain: "source" or "target"
        returns: (batch,) predicted scalar ratings
        """
        u = self.user_emb(users)  # (batch, user_emb_dim)
        # domain-specific adapter: lets the model learn per-domain user behavior while sharing base info
        if domain == "source":
            u_dom = self.user_adapter_src(u)
            i_feat = self.item_feat_src(items)
            x = torch.cat([u_dom, i_feat], dim=-1)
            out = self.mlp_src(x).squeeze(-1)
        elif domain == "target":
            u_dom = self.user_adapter_tgt(u)
            i_feat = self.item_feat_tgt(items)
            x = torch.cat([u_dom, i_feat], dim=-1)
            out = self.mlp_tgt(x).squeeze(-1)
        else:
            raise ValueError("domain must be 'source' or 'target'")
        return out
# ...existing code...

In [ ]:
# Example usage (assumes the named variables exist in the notebook):
# - source_item_feature_vectors, target_item_feature_vectors: arrays/tensors of item features
# - source_loader, target_loader: DataLoader objects producing (user_idx, item_idx, rating)


In [ ]:
model = NCF(n_users=n_users,
                 source_item_features=source_item_feature_vectors,
                 target_item_features=target_item_feature_vectors,
                 user_emb_dim=64,
                 mlp_layers=(128,64,32),
                 dropout=0.2,
                 freeze_item_features=True)

train_ncf(model, source_loader, target_loader, epochs=50, lr=1e-3)
print("Target RMSE:", evaluate_ncf(model, target_loader, domain="target"))

Epoch 1/50 avg MSE: 3.040604
Epoch 2/50 avg MSE: 1.408292
Epoch 3/50 avg MSE: 1.271468
Epoch 4/50 avg MSE: 1.212633
Epoch 5/50 avg MSE: 1.175297
Epoch 6/50 avg MSE: 1.136314
Epoch 7/50 avg MSE: 1.113781
Epoch 8/50 avg MSE: 1.085496
Epoch 9/50 avg MSE: 1.059976
Epoch 10/50 avg MSE: 1.043349
Epoch 11/50 avg MSE: 1.024997
Epoch 12/50 avg MSE: 1.004933
Epoch 13/50 avg MSE: 0.988168
Epoch 14/50 avg MSE: 0.970328
Epoch 15/50 avg MSE: 0.953801
Epoch 16/50 avg MSE: 0.931623
Epoch 17/50 avg MSE: 0.916105
Epoch 18/50 avg MSE: 0.898908
Epoch 19/50 avg MSE: 0.881925
Epoch 20/50 avg MSE: 0.862051
Epoch 21/50 avg MSE: 0.847788
Epoch 22/50 avg MSE: 0.831449
Epoch 23/50 avg MSE: 0.812696
Epoch 24/50 avg MSE: 0.797843
Epoch 25/50 avg MSE: 0.781795
Epoch 26/50 avg MSE: 0.762961
Epoch 27/50 avg MSE: 0.748274
Epoch 28/50 avg MSE: 0.734222
Epoch 29/50 avg MSE: 0.716033
Epoch 30/50 avg MSE: 0.698651
Epoch 31/50 avg MSE: 0.688903
Epoch 32/50 avg MSE: 0.675081
Epoch 33/50 avg MSE: 0.662427
Epoch 34/50 avg MSE

In [25]:
preds = model.forward(tensor_target_users.long(), tensor_target_items.long(), 'target')

In [26]:
bin_preds = [int(e >= 3) for e in preds]
bin_ratings = [int(e >= 3) for e in tensor_target_ratings]

In [27]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

acc = accuracy_score(bin_ratings, bin_preds)
conf = precision_recall_fscore_support(bin_ratings, bin_preds, average='binary')

In [28]:
print("acc:", acc)
print("prec:", conf[0])
print("recall:", conf[1])
print("f1:", conf[2])

acc: 0.9506054469562343
prec: 0.9757467799397095
recall: 0.969846371758553
f1: 0.9727876287533127
